In [1]:
cd /Users/oleg.vlasovetc/Public/GGLasso

/Users/oleg.vlasovetc/Public/GGLasso


In [2]:
import sys
import time
import numpy as np
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance
from sklearn import set_config
set_config(print_changed_only=False)

from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import block_SGL
from gglasso.helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from gglasso.helper.model_selection import single_grid_search
from benchmarks import models_to_dict, sklearn_time_benchmark, admm_time_benchmark, model_solution
from benchmarks import hamming_distance, distance_benchmark, network_generation

from regain.covariance import GraphicalLasso as rg_GL

In [4]:
S, Theta, X = network_generation(p_list=[100], N_list=[200], K=5, M=2)

In [6]:
Omega_0 = np.eye(S[0].shape[0])
lambda1=0.01
# tol_list = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
rtol_list = [1e-2, 1e-4]
enet_list = [0.1]

# Model solution Z

In [8]:
Z, Z_time = model_solution(model="sklearn", X=X, lambda1=lambda1, max_iter=50000, tol=1e-10, enet=0.1)
print(Z_time)

sklearn GraphicalLasso(alpha=0.01, assume_centered=True, enet_tol=0.1, max_iter=50000,
               mode='cd', tol=1e-10, verbose=False)
00:00:00.99


# Solution Z_i

In [9]:
models = models_to_dict(models=["sklearn", "regain"], lambda1=lambda1, tol_list=tol_list, rtol_list=rtol_list, 
                        enet_list=enet_list, 
                        max_iter=50000)

In [10]:
sk_time, sk_accuracy, Z_sk = sklearn_time_benchmark(models, X=X, Z=Z, n_iter=5)

regain_tol_1e-07_rtol_0.0001: 100%|██████████| 5/5 [00:01<00:00,  4.20it/s]


In [7]:
admm_time, admm_accuracy, Z_admm = admm_time_benchmark(S=S, Omega_0=Omega_0, Z=Z, lambda1=lambda1, 
                                                   method_list=["single", "block"],
                                                   stop_list=['boyd', 'kkt'], 
                                                   tol_list=tol_list, 
                                                   rtol_list=rtol_list,
                                                   n_iter=1 + 5)

single-kkt_tol_0.0001_rtol_0.01:   0%|          | 0/6 [00:00<?, ?it/s]

ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.
ADMM terminated after 8 iterations with status: optimal.


single-kkt_tol_0.0001_rtol_0.01:  17%|█▋        | 1/6 [00:00<00:03,  1.28it/s]

ADMM terminated after 67 iterations with status: optimal.


single-kkt_tol_0.0001_rtol_0.01:  33%|███▎      | 2/6 [00:01<00:02,  1.59it/s]

...

ADMM terminated after 157 iterations with status: optimal.


block-kkt_tol_1e-07_rtol_0.0001:  83%|████████▎ | 5/6 [00:04<00:00,  1.05it/s]

ADMM terminated after 157 iterations with status: optimal.


block-kkt_tol_1e-07_rtol_0.0001: 100%|██████████| 6/6 [00:05<00:00,  1.15it/s]

ADMM terminated after 157 iterations with status: optimal.


# Data formating

In [15]:
sparsity_sk = distance_benchmark(Theta, Z_sk, t=1e-5)
sparsity_admm = distance_benchmark(Theta, Z_admm, t=1e-5)

In [16]:
df_sk = pd.DataFrame(data={'name': sk_time.keys(), 'time': sk_time.values(), "accuracy": sk_accuracy.values(),
                          "hamming": sparsity_sk.values()})

df_admm = pd.DataFrame(data={'name': admm_time.keys(), 'time': admm_time.values(), "accuracy": admm_accuracy.values(),
                            "hamming": sparsity_admm.values()})

df = pd.concat([df_sk, df_admm])
df.head()

,name,time,accuracy,hamming
0,sklearn_tol_0.0001_enet_0.1,0.228450,3.146413e-03,8262
1,sklearn_tol_1e-05_enet_0.1,0.612504,1.673918e-05,8258
2,sklearn_tol_1e-06_enet_0.1,0.705758,1.492459e-06,8258
3,sklearn_tol_1e-07_enet_0.1,0.668444,1.747504e-07,8258
4,regain_tol_0.0001_rtol_0.01,0.129911,1.145834e-02,8272


In [24]:
df['split'] = df['name'].str.split('_')
df[["method", "tol_str", "tol", "rtol_str", "rtol"]] = pd.DataFrame(df['split'].tolist(), index=df['split'].index)
df = df.drop(['split', "tol_str", "rtol_str"], axis=1)
df.head()

,name,time,accuracy,sparsity,method,tol,rtol
0,sklearn_tol_0.0001_enet_0.1,58.703270,1.209231e-05,0.041172,sklearn,0.0001,0.1
1,sklearn_tol_1e-05_enet_0.1,100.285248,5.379661e-06,0.041172,sklearn,1e-05,0.1
2,sklearn_tol_1e-06_enet_0.1,112.136015,3.223257e-08,0.041172,sklearn,1e-06,0.1
3,sklearn_tol_1e-07_enet_0.1,502.651721,4.956103e-09,0.041172,sklearn,1e-07,0.1
4,regain_tol_0.0001_rtol_0.01,9.431963,1.433152e-02,0.039145,regain,0.0001,0.01


Here in sklearn, "rtol" corresponds to "enet_tol", see [sklearn.covariance.graphical_lasso](https://scikit-learn.org/stable/modules/generated/sklearn.covariance.graphical_lasso.html)

In [25]:
fig = px.line(df, x="time", y="accuracy", text = "name", color= "method", 
              log_y = True, 
              labels={
                     "time": "Time, s",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()

In [12]:
fig = px.line(df, x="tol", y="accuracy", color= "method", 
              log_y = True,
              labels={
                     "tol": "Tolerance rate",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()

# Sparsity benchmark

In [13]:
fig = go.Figure(data=go.Heatmap(
        z=df["sparsity"],
        x=df["method"],
        y=df["tol"],
        colorscale='Viridis'))

fig.update_layout(
    title='Sparsity benchmarks',
    xaxis_nticks=36)

fig.show()